In [1]:
# declare a list tasks whose products you want to use as inputs
upstream = ['fetch_all_securities_list', 'fetch_snp500_securities_list']

In [2]:
# Parameters
sec_base_url = "https://data.sec.gov/submissions/CIK{cik_str}.json"
sample_pct = 1
snp_filter = False
upstream = {
    "fetch_snp500_securities_list": {
        "nb": "/Users/aiujdm2/market_watch/output/notebooks/fetch_snp500_securities_list.ipynb",
        "data": "/Users/aiujdm2/market_watch/output/data/raw/snp500_securities_list.csv",
    },
    "fetch_all_securities_list": {
        "nb": "/Users/aiujdm2/market_watch/output/notebooks/fetch_all_securities_list.ipynb",
        "data": "/Users/aiujdm2/market_watch/output/data/raw/all_securities_list.csv",
    },
}
product = {
    "nb": "/Users/aiujdm2/market_watch/output/notebooks/fetch_securities.ipynb",
    "data": "/Users/aiujdm2/market_watch/output/data/raw/securities.csv",
}


In [3]:
import pandas as pd
import json
import requests
import time 
from pathlib import Path
from tqdm import tqdm
from src.utils import preprocess_text

[nltk_data] Downloading package wordnet to /Users/aiujdm2/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aiujdm2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/aiujdm2/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/Users/aiujdm2/.local/share/virtualenvs/market_watch-dtlP-L11/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Get list of publicaly traded companies, ticker and associated cik ID. We will use cik ID to pull data from SEC's [DATA APIS](https://www.sec.gov/edgar/sec-api-documentation)

In [4]:
traded_company_df = pd.read_csv(upstream["fetch_all_securities_list"]["data"], index_col=0)
snp500_company_df = pd.read_csv(upstream["fetch_snp500_securities_list"]["data"], index_col=0)

# SEC data API needs cik number to be padded by 0's to make it a length of 10 string
traded_company_df['cik'] = traded_company_df.cik_str
traded_company_df.cik_str = traded_company_df.cik_str.astype(str).str.zfill(10)
snp500_company_df['cik_str'] = snp500_company_df.CIK.astype(str).str.zfill(10)

Download dataset for publicly traded companies incl. metadata like previous name etc. See the [DEVELOPER FAQ](https://www.sec.gov/os/webmaster-faq#developers) to get more infomration about rate limit and API call requirements.

In [5]:
# snp_filter parameter controls if we want to fetch all master publicly traded securities or just the snp500 ones
if snp_filter == True:
    traded_companies = snp500_company_df.cik_str
else:
    # sample_pct is helpful parameter to help speed up development by selecting only a fraction of securities out of 12,227 master securities       
    traded_companies = traded_company_df.cik_str.sample(frac=sample_pct)

Let's Fetch SEC submission data. This API has 10 request per second limit. so it will take about 1 hour to finish.preprocess_text

In [6]:
# SEC data API requires explicit setting of User-agent as email address, otehrwise we will get 404 error
headers = {
  'User-Agent': 'vgaurav@umich.edu'
}

df_lst = []
counter = 0
#  let's reuse connection pool by using same session.
session = requests.Session()

for cik_str in tqdm(traded_companies):
    url = sec_base_url.format(cik_str = cik_str)
    # we could have used following code, but it does not work due to nested json format of payload
    # pd.read_json(url, stosample_pctptions = {'User-Agent': 'vgaurav@umich.edu'})
    response = session.get(url, headers=headers)
    data = json.loads(response.text)
    df_lst.append(pd.json_normalize(data))
    counter += 1
    # SEC API restricts 10 API request per second, so we will use counter and time to meet this restriction
    if counter % 10 == 0:
        time.sleep(1)
        
securities_df = pd.concat(df_lst)

  0%|                                                  | 0/9193 [00:00<?, ?it/s]

  0%|                                          | 1/9193 [00:00<26:18,  5.82it/s]

  0%|                                          | 3/9193 [00:00<18:13,  8.40it/s]

  0%|                                          | 5/9193 [00:00<17:49,  8.59it/s]

  0%|                                          | 6/9193 [00:00<17:21,  8.82it/s]

...

100%|██████████████████████████████████████▉| 9190/9193 [36:02<00:01,  2.76it/s]

100%|██████████████████████████████████████▉| 9191/9193 [36:02<00:00,  3.33it/s]

100%|██████████████████████████████████████▉| 9192/9193 [36:02<00:00,  4.01it/s]

100%|███████████████████████████████████████| 9193/9193 [36:03<00:00,  4.74it/s]

100%|███████████████████████████████████████| 9193/9193 [36:03<00:00,  4.25it/s]

In [7]:
securities_df.cik = securities_df.cik.astype(int)
securities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9193 entries, 0 to 0
Data columns (total 47 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   cik                                           9193 non-null   int64 
 1   entityType                                    9193 non-null   object
 2   sic                                           9193 non-null   object
 3   sicDescription                                9193 non-null   object
 4   insiderTransactionForOwnerExists              9193 non-null   int64 
 5   insiderTransactionForIssuerExists             9193 non-null   int64 
 6   name                                          9193 non-null   object
 7   tickers                                       9193 non-null   object
 8   exchanges                                     9193 non-null   object
 9   ein                                           9179 non-null   object
 10  des

In [8]:
traded_company_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9193 entries, 0 to 12122
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   cik_str  9193 non-null   object
 1   ticker   9193 non-null   object
 2   title    9193 non-null   object
 3   cik      9193 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 359.1+ KB


In [9]:
merged_df = securities_df.merge(traded_company_df, on='cik', how='inner')
# merged_df = merged_df.dropna(subset=['tickers'])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9193 entries, 0 to 9192
Data columns (total 50 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   cik                                           9193 non-null   int64 
 1   entityType                                    9193 non-null   object
 2   sic                                           9193 non-null   object
 3   sicDescription                                9193 non-null   object
 4   insiderTransactionForOwnerExists              9193 non-null   int64 
 5   insiderTransactionForIssuerExists             9193 non-null   int64 
 6   name                                          9193 non-null   object
 7   tickers                                       9193 non-null   object
 8   exchanges                                     9193 non-null   object
 9   ein                                           9179 non-null   object
 10  

In [10]:
snp500_company_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 503
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Symbol                 500 non-null    object
 1   Security               500 non-null    object
 2   SEC filings            500 non-null    object
 3   GICS Sector            500 non-null    object
 4   GICS Sub-Industry      500 non-null    object
 5   Headquarters Location  500 non-null    object
 6   Date first added       455 non-null    object
 7   CIK                    500 non-null    int64 
 8   Founded                500 non-null    object
 9   cik_str                500 non-null    object
dtypes: int64(1), object(9)
memory usage: 43.0+ KB


In [11]:
# left join ensures we populate fields from snp 500 df where applicable
merged_securities_df = merged_df.merge(snp500_company_df, left_on='cik', right_on='CIK', how='left')
merged_securities_df = merged_securities_df[['cik','ticker', 'exchanges', 'ein', 'name','formerNames', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'sic', 'sicDescription', 'Headquarters Location']]
merged_securities_df.columns = ['cik','ticker', 'exchanges', 'ein', 'full_name','former_names', 'short_name', 'gics_sector', 'gics_sub_industry', 'sic','sic_description', 'headquarters_location']
merged_securities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9193 entries, 0 to 9192
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   cik                    9193 non-null   int64 
 1   ticker                 9193 non-null   object
 2   exchanges              9193 non-null   object
 3   ein                    9179 non-null   object
 4   full_name              9193 non-null   object
 5   former_names           9193 non-null   object
 6   short_name             499 non-null    object
 7   gics_sector            499 non-null    object
 8   gics_sub_industry      499 non-null    object
 9   sic                    9193 non-null   object
 10  sic_description        9193 non-null   object
 11  headquarters_location  499 non-null    object
dtypes: int64(1), object(11)
memory usage: 933.7+ KB


In [12]:
output_file_path = product['data']
print(len(merged_securities_df))

9193


In [13]:
merged_securities_df.to_csv(output_file_path)
print(f"Saved file {output_file_path}")

Saved file /Users/aiujdm2/market_watch/output/data/raw/securities.csv


In [14]:
merged_securities_df.sample(10)

,cik,ticker,exchanges,ein,full_name,former_names,short_name,gics_sector,gics_sub_industry,sic,sic_description,headquarters_location
4103,884713,PRMW,[NYSE],980154711,Primo Water Corp /CN/,"[{'name': 'COTT CORP /CN/', 'from': '1995-02-0...",NaN,NaN,NaN,2086,Bottled & Canned Soft Drinks & Carbonated Waters,NaN
2020,1140411,PCQ,[NYSE],134174445,PIMCO CALIFORNIA MUNICIPAL INCOME FUND,[],NaN,NaN,NaN,,,NaN
3691,1841125,GAMC,"[Nasdaq, Nasdaq, Nasdaq]",861256660,Golden Arrow Merger Corp.,[],NaN,NaN,NaN,6770,Blank Checks,NaN
3162,1121142,MAXR,[NYSE],980544351,Maxar Technologies Inc.,"[{'name': 'Maxar Technologies Ltd.', 'from': '...",NaN,NaN,NaN,3663,Radio & Tv Broadcasting & Communications Equip...,NaN
2073,1828185,VACC,[Nasdaq],000000000,Vaccitech plc,"[{'name': 'Vaccitech Ltd', 'from': '2020-12-23...",NaN,NaN,NaN,2834,Pharmaceutical Preparations,NaN
4136,1307748,IVT,[NYSE],342019608,InvenTrust Properties Corp.,"[{'name': 'Inland American Real Estate Trust, ...",NaN,NaN,NaN,6798,Real Estate Investment Trusts,NaN
1823,1885768,INRX,[],834622006,"Intrinsic Medicine, Inc.",[],NaN,NaN,NaN,2836,"Biological Products, (No Diagnostic Substances)",NaN
910,1781397,EEIQ,[Nasdaq],461605475,Elite Education Group International Ltd,[],NaN,NaN,NaN,8200,Services-Educational Services,NaN
8707,850918,GWLIF,[OTC],000000000,GREAT-WEST LIFECO INC.,"[{'name': 'GREAT WEST LIFECO INC', 'from': '20...",NaN,NaN,NaN,6311,Life Insurance,NaN
6738,1528308,IINX,[OTC],450713638,"IONIX TECHNOLOGY, INC.","[{'name': 'CAMBRIDGE PROJECTS INC.', 'from': '...",NaN,NaN,NaN,3679,"Electronic Components, NEC",NaN


In [15]:
del securities_df, traded_company_df, traded_companies, snp500_company_df, merged_securities_df